In [2]:
import requests
from bs4 import BeautifulSoup
import json


In [3]:
#女優一覧（あ）
#url1 = "https://www.dmm.co.jp/digital/videoa/-/actress/=/keyword=i/" #max_page 6
#url1 = "https://www.dmm.co.jp/digital/videoa/-/actress/=/keyword=a/" #max_page 12
url1 = "https://www.dmm.co.jp/digital/videoa/-/actress/=/keyword=re/" #max_page 1

In [4]:
#ジャンル（OL)
url2 = "https://www.dmm.co.jp/digital/videoa/-/list/=/article=keyword/id=1001/" #max_page 89
#ジャンル（受付嬢）
#url2 = "https://www.dmm.co.jp/digital/videoa/-/list/=/article=keyword/id=6942/" #max_page 1

In [5]:
#作品ページ（篠田ゆう）
url3 = "https://www.dmm.co.jp/digital/videoa/-/list/=/article=actress/id=1008785/"

In [6]:
url4 = "https://www.dmm.co.jp/digital/videoa/-/list/=/article=actress/id=1008785/"

In [7]:
url5 = "https://www.dmm.co.jp/digital/videoa/-/detail/=/cid=mrhp00004/?dmmref=actress_1008785&i3_ref=list&i3_ord=33"

In [16]:
def actor_max_page(soup2):
    #５０音キーワード別女優総人数：total_number_of_actor
    total_number_of_actor= soup2.find('div', class_ = "list-boxcaptside list-boxpagenation group").p.text.replace('　', '').split('人')[0]
    max_page_number = soup2.find('div', class_ = "list-boxcaptside list-boxpagenation group").p.text.replace('　', '').split('全')[1].split('ペ')[0]
    return max_page_number

In [17]:
def genre_sakuhi_maxpage(soup2):
    #ジャンルページは、「10,581タイトル中　1～120タイトル　1ページ目を表示」で全ページ数の表示がないのでタイトル数//１２０で切り上げ割り算でページ数を取得する
    #.replace(',', '')はタイトル数が４桁になると「1,000」とカンマが入るので削除している。int()に変換できないので。この記述がタイトル数３桁でエラーを発生することはない。
    sakuhin_title_number = int(soup2.find('div', class_ ="list-boxcaptside list-boxpagenation").p.text.replace('　', '').split('タ')[0].replace(',', ''))
    # 121//120は切り捨て割り算、期待値は切り上げ割り算の結果で下記のようになる。
    max_page_number = -(-sakuhin_title_number // 120)
    return max_page_number

In [18]:
def soup2(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    res2 = requests.get(soup.find('a', class_ = "ageCheck__link ageCheck__link--r18")['href'])

    soup2 = BeautifulSoup(res2.text, 'html.parser')
    
    if "d/i" in url:
        print("ジャンルページです")
        #ジャンルページ用の関数を実行
        max_page = genre_sakuhi_maxpage(soup2)
        url_kind = "genre"
        baseurl = "https://www.dmm.co.jp/digital/videoa/-/list/=/article=keyword/id=1001/page="
    elif "a/-/a" in url:
        print("５０音ページです")
        url_kind ="50on"
        max_page =  actor_max_page(soup2) 
        baseurl = "https://www.dmm.co.jp/digital/videoa/-/actress/=/keyword=a/page="
        #５０音ページ用の関数を実行
    elif "t&" in url:
        print("ビデオページです")
        url_kind = "video"
        #ビデオページ用の関数を実行

    elif "s/i" in url:
        print("出演一覧ページ")
        url_kind = "syutuen_itiran"
        max_page = genre_sakuhi_maxpage(soup2)
        baseurl = "https://www.dmm.co.jp/digital/videoa/-/list/=/article=actress/id=1008887/page="
        #出演一覧ページ用の関数を実行

    return soup2, url_kind, max_page+1, baseurl

In [19]:
soup9, url_kind, max_page, baseurl= soup2(url2)

ジャンルページです


In [26]:
#ページネーション分のURLを作成する
def make_url(max_page, baseurl):
    url_lists = []
    for page in range(1, max_page):
        (baseurl + str(page) + "/")
        url_lists.append(baseurl + str(page) + "/")
    print(url_lists)

In [25]:
make_url(max_page, baseurl)

['https://www.dmm.co.jp/digital/videoa/-/list/=/article=keyword/id=1001/page=1/', 'https://www.dmm.co.jp/digital/videoa/-/list/=/article=keyword/id=1001/page=2/', 'https://www.dmm.co.jp/digital/videoa/-/list/=/article=keyword/id=1001/page=3/', 'https://www.dmm.co.jp/digital/videoa/-/list/=/article=keyword/id=1001/page=4/', 'https://www.dmm.co.jp/digital/videoa/-/list/=/article=keyword/id=1001/page=5/', 'https://www.dmm.co.jp/digital/videoa/-/list/=/article=keyword/id=1001/page=6/', 'https://www.dmm.co.jp/digital/videoa/-/list/=/article=keyword/id=1001/page=7/', 'https://www.dmm.co.jp/digital/videoa/-/list/=/article=keyword/id=1001/page=8/', 'https://www.dmm.co.jp/digital/videoa/-/list/=/article=keyword/id=1001/page=9/', 'https://www.dmm.co.jp/digital/videoa/-/list/=/article=keyword/id=1001/page=10/', 'https://www.dmm.co.jp/digital/videoa/-/list/=/article=keyword/id=1001/page=11/', 'https://www.dmm.co.jp/digital/videoa/-/list/=/article=keyword/id=1001/page=12/', 'https://www.dmm.co.jp/d

In [13]:
def sakuhin():
    base_url = "baseurl:https://www.dmm.co.jp/digital/videoa/-/list/=/article=actress/id=1008887/page="
    sakuhin_lists = []
    #ページネーション処理
    for row in range(1, max_page): #index 1から開始指定
        #print(row)
        #print(url + "page=" + str(row))
        res3 = requests.get(url + "page=" + str(row))
        soup4 = BeautifulSoup(res3.text, 'html.parser')
        res4 = requests.get(soup4.find('a', class_="ageCheck__link ageCheck__link--r18").get('href'))
        soup5 = BeautifulSoup(res4.text, 'html.parser')
        actress = soup5.find('ul', class_ = 'd-item act-box-100 group')
        #print(actress.select('span')[0].text)
        #配列内に配列を追記

        act_list = [furigana, act_name, syutuen_su, act_gazou_url, act_base_url]
        act_lists.pend(act_list)
    return act_lists

In [9]:
soup9, url_kind = soup2(url4)

出演一覧ページ


In [10]:
soup9.title

<title>篠田ゆう(しのだゆう) - エロ動画・アダルトビデオ - FANZA動画</title>

In [12]:
if url_kind == "syutuen_itiran":
    su = sakuhin(soup9)

TypeError: sakuhin() missing 1 required positional argument: 'url'